# Numpy - Beginner (TF2)

TensorFlow2 공식 홈페이지에 나와 있는 코드들을 수행해 보면서 빠르게 TensorFlow2를 익혀 봅니다.

https://www.tensorflow.org/tutorials/load_data/numpy

##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil, functools
import collections, pathlib, re, string

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import tensorflow as tf
tf.random.set_seed(rseed)
tf.keras.backend.set_floatx('float32') # keras default float type 설정

import tensorflow_hub as tfhub
import tensorflow_datasets as tfds
import tensorflow_text as tftext

import kerastuner as kt

import seaborn as sns

import PIL
import PIL.Image

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"tensorflow ver={tf.__version__}")
print(f"tensorflow execuring eagerly={tf.executing_eagerly()}")
print(f"tensorflow GPU={'True' if tf.config.experimental.list_physical_devices('GPU') else 'False'}")
print(f"keras tuner ver={kt.__version__}")
print(f"senborn ver={sns.__version__}")
print(f"PIL ver={PIL.__version__}")

python ver=3.7.6 (default, Nov 21 2020, 22:51:13) 
[Clang 12.0.0 (clang-1200.0.32.27)]
pandas ver=1.0.5
numpy ver=1.19.5
tensorflow ver=2.4.1
tensorflow execuring eagerly=True
tensorflow GPU=False
keras tuner ver=1.0.2
senborn ver=0.10.0
PIL ver=7.1.2


## Load NumPy data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/numpy"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/numpy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/numpy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/numpy.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
*

____
This tutorial provides an example of loading data from NumPy arrays into a `tf.data.Dataset`.

This example loads the MNIST dataset from a `.npz` file. However, the source of the NumPy arrays is not important.


## Setup

### Load from `.npz` file

In [3]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('/tmp/mnist.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

11493376/11490434 [==============================] - 0s 0us/step


## Load NumPy arrays with `tf.data.Dataset`

Assuming you have an array of examples and a corresponding array of labels, pass the two arrays as a tuple into `tf.data.Dataset.from_tensor_slices` to create a `tf.data.Dataset`.

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

## Use the datasets

### Shuffle and batch the datasets

In [5]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

### Build and train a model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [7]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 1s 1ms/step - loss: 8.3540 - sparse_categorical_accuracy: 0.8197
Epoch 2/10
938/938 [==============================] - 1s 1ms/step - loss: 0.5988 - sparse_categorical_accuracy: 0.9186
Epoch 3/10
938/938 [==============================] - 1s 1ms/step - loss: 0.4038 - sparse_categorical_accuracy: 0.9413
Epoch 4/10
938/938 [==============================] - 1s 1ms/step - loss: 0.3484 - sparse_categorical_accuracy: 0.9528
Epoch 5/10
938/938 [==============================] - 1s 1ms/step - loss: 0.2935 - sparse_categorical_accuracy: 0.9577
Epoch 6/10
938/938 [==============================] - 1s 1ms/step - loss: 0.2743 - sparse_categorical_accuracy: 0.9632
Epoch 7/10
938/938 [==============================] - 1s 1ms/step - loss: 0.2373 - sparse_categorical_accuracy: 0.9664
Epoch 8/10
938/938 [==============================] - 1s 1ms/step - loss: 0.2404 - sparse_categorical_accuracy: 0.9678
Epoch 9/10
938/938 [============================

In [8]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 761us/step - loss: 0.6825 - sparse_categorical_accuracy: 0.9595


[0.6825053095817566, 0.9595000147819519]